In [25]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from nets.cut import Cut
from pytorch_lightning import Trainer
from data_loader_gaelle import ConcatDataset,ImageDataset, LotusDataModule, ConcatDataModule, LotusTrainTransforms2
import pandas as pd
import numpy as np 

# from callbacks import logger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import NeptuneLogger
from pytorch_lightning.strategies.ddp import DDPStrategy

from monai.transforms import (  
    LoadImaged,
    Compose,
    Resize,
    RandZoomd,
    RandRotated,
    RandAffined,
    ToTensord
)   
import math


ImportError: cannot import name 'LotusTrainTransforms2' from 'data_loader_gaelle' (/home/luciacev/Documents/Gaelle/MRI_to_CBCT/mri_to_cbct/data_loader_gaelle.py)

In [14]:
df_train_cbct = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/train.csv") 
df_val_cbct = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/valid.csv")  
df_test_cbct = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/test.csv")

df_train_mri = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/train.csv")
df_val_mri = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/valid.csv")   
df_test_mri = pd.read_csv("/home/luciacev/Documents/Gaelle/Data/MultimodelReg_2/MRI_to_CBCT/training_CBCT/test.csv") 


print("df_train_cbct : ",df_train_cbct)

df_train_cbct :      index                  img_fn group  ignore_size_info  label
0       0  train/B040_cropped.nii     .                 0      1
1       1  train/B041_cropped.nii     .                 0      2
2       2  train/M001_cropped.nii     .                 0      3
3       3  train/M002_cropped.nii     .                 0      4
4       4  train/M003_cropped.nii     .                 0      5
..    ...                     ...   ...               ...    ...
59     59  train/M061_cropped.nii     .                 0     60
60     60  train/M062_cropped.nii     .                 0     61
61     61  train/M066_cropped.nii     .                 0     62
62     62  train/M067_cropped.nii     .                 0     63
63     63  train/M073_cropped.nii     .                 0     64

[64 rows x 5 columns]


In [15]:
transform = transforms.Compose(
    [   
        # EnsureChannelFirstd(keys=['img', 'seg'], channel_dim=-1),
        LoadImaged(keys=['img_mri', 'img_cbct']),
        RandZoomd(keys=['img_mri', 'img_cbct'], min_zoom=0.8, max_zoom=1.1, mode=['area', 'nearest'], prob=0.9, padding_mode='constant'),
        RandRotated(keys=['img_mri', 'img_cbct'], range_x=math.pi, mode=['bilinear', 'nearest'], prob=1.0),
        RandAffined(keys=['img_mri', 'img_cbct'], prob=0.8, shear_range=(0.1, 0.1), mode=['bilinear', 'nearest'], padding_mode='zeros'),
        ToTensord(keys=['img_mri', 'img_cbct'])
    ]

)

In [16]:
train_transform_mri = LotusTrainTransforms()
valid_transform_mri = LotusTrainTransforms()
MRI_data = LotusDataModule(df_train_mri, df_val_mri, df_test_mri, mount_point=".", batch_size=2, num_workers=4, img_column="img_path", seg_column="seg_path", train_transform=train_transform_mri, valid_transform=valid_transform_mri, test_transform=valid_transform_mri, drop_last=False)

MRI_data.setup()


train_transform_cbct = ImageDataset()
valid_transform_cbct = ImageDataset()
CBCT_data = LotusDataModule(df_train_cbct, df_val_cbct, df_test_cbct, mount_point=".", batch_size=2, num_workers=4, img_column="img_path", seg_column="seg_path", train_transform=train_transform_cbct, valid_transform=valid_transform_cbct, test_transform=valid_transform_cbct, drop_last=False)
CBCT_data.setup()

NameError: name 'LotusTrainTransforms' is not defined